In [201]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity

In [343]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

In [344]:
keys, content_list = contents.keys(), contents.values

In [ ]:
all_doc = content_list# (train_dataset.contents)
sentenece_arr = [ jieba.lcut(sen) for sen in all_doc]

In [ ]:
from gensim import corpora
word_list = sentenece_arr
dictionary = corpora.Dictionary(word_list)
new_corpus = [ dictionary.doc2bow(text) for text in word_list]

In [ ]:
doc_dict = {}
for idx, text in enumerate(new_corpus):
    all_words = ([i[0] for i in text])
    
    for word in all_words:
        if word in doc_dict.keys():
            doc_dict[word].append(idx)
        else:
            doc_dict[word]= [idx]

In [ ]:
from sklearn.model_selection import train_test_split
batch_size = 100000
folder = 'news_data_1/'
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

In [ ]:
total_scores = list()

for test_id , text_q in enumerate(test_query):
    print( test_id ,text_q)
    text = jieba.lcut(text_q)
    text_bow = dictionary.doc2bow(text) 
    keys, values = zip(*tfidf[text_bow])
    
    for cnt, key in enumerate(keys):
        if cnt == 0:
            queried_doc = set(doc_dict[key])
        else:
            queried_doc = set(doc_dict[key]) | queried_doc
    
    cosine_sim_score = list()
    for cnt, doc in enumerate(queried_doc):
        tfidf_score = np.zeros((len(keys)))
        result = sentenece_arr[doc]
        result = dictionary.doc2bow(result)
        result_dict = dict(tfidf[result])
        
        for idx, i in enumerate(keys):
            if i in result_dict.keys():
                tfidf_score[idx] = result_dict[i]
        
        tfidf_score, values  = np.array(tfidf_score).reshape(1,-1), np.array(values).reshape(1,-1)
        score = cosine_similarity(tfidf_score, values)
        cosine_sim_score.append((doc, score[0][0]))
        
    total_scores.append(cosine_sim_score)

In [ ]:
search_result = np.zeros((20,300))
for cnt,i in enumerate(total_scores):
    keys ,values = zip(*sorted(i, key= lambda x: x[1])[::-1][:300])
    search_result[cnt] = keys

In [262]:
search_result=search_result.astype(np.int)

In [263]:
# results = results.reshape(20, -1)
# search_result = np.flip(np.argsort(results, axis=1), axis=1)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(300):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i]

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))
    
fname = 'simple7.csv'
df.to_csv('output/' + fname, index=False)

In [324]:
idx=search_result[1][0]
content_list[search_result[1][0]]

'有民眾於國發會的公共政策網路參與平臺提議「提升全民行車觀念，以提升全民行車環境品質，解除禁行機慢車道，解除強制二段式左轉」，交通部認為涉及地方政府道路交通工程配置規畫事宜，經函徵各地方政府意見，多數認不宜貿然廢除，應尊重地方政府的權責。\n交通部表示，因涉地方政府交通管理規畫權責，函徵全國各縣市政府、內政部警政署、公路總局及運輸研究所等單位意見，並於5月26日召開會議，邀請提案人及相關單位共同討論。中央主管機關應尊重各地道路主管機關係基於安全及效率之交通管理規劃。鑑於現行法規已留給地方政府彈性管理之空間，應尚無需研修相關法規，地方政府可依現場道路交通環境情況，因地制宜綜合評估是否設置或取消禁行機車道及兩段式左轉相關標誌標線。\n目前已有多個縣市開始試辦及追蹤實施後之事故改善情形，並逐步擴大檢討，例如臺北市在42處路段試辦取消第三車道禁行機車，結果25處事故改善，17處惡化，因沒有顯著效果，目前實作仍以個案方式檢討施行。\n交通部運輸研究所亦已就「路段共享道路空間」及「路口減少轉向衝突」之改善目標，研提具安全性及全國性可適用之規劃原則可供各縣市政府參考，該部將督同公路總局及各直轄市、縣（市）政府因地制宜檢討各級道路配置，以提供更友善的交通環境供不同族群用路人合理使用道路。\n(中時 )'

In [329]:
contents[news_urls.values[idx][1]]

'喧騰甚久的陳水扁保外就醫案雖然歷經一波三折，但終究隨著阿扁獲准保外就醫並回到家中，似乎暫告一個段落，但是否真能讓台灣社會回歸平靜，進而藍綠走向和解，關鍵還在於陳水扁自己，他若能做到諸法皆空，才會真的自由自在。\n曾有民進黨人形容阿扁是「標準的政治動物」，他的興趣不多，也少有休閒娛樂，腦中思考的、心中所想的都是政治上的折衝協調、縱橫捭闔，不折不扣的政治人。\n所以，陳水扁在剛入監但病情尚未惡化之前，雖然人在坐牢，卻時時與外界的政治情勢相連結，三不五時透過探監的人傳達阿扁的政治判斷或評論，有時還對民進黨下起指導棋，讓不少人深感阿扁的不甘寂寞。\n基於人道與台灣社會的和諧，社會超越藍綠的獲致最大公約數，多數贊成讓阿扁保外就醫，但阿扁現在也應思考，從總統之尊的天堂到階下囚的地獄，現又再回到溫暖的家中，他是否真能像聖嚴法師說的「放下它」，才是核心。\n當阿扁心中能夠真正放下，才能真正得到解脫，台灣社會方能放下這個桎梏。\n(旺報)'

In [ ]:
import pickle 
with  open('test.pickle') as file:
    pickle.dump(, file)